# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [8]:
%matplotlib inline
import numpy as np   

# Import Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
import pandas as pd    

import matplotlib.pyplot as plt   

import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
# Import library for metrics
from sklearn.metrics import  r2_score
import pickle

### 2. Load the dataset

In [9]:
car_data=pd.read_csv(r'/Users/moni/Downloads/car+data.csv')
car_data.head(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [10]:
car_data.shape

(301, 9)

In [11]:
car_data.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [12]:
len(car_data.duplicated())

301

In [13]:
car_data.drop_duplicates(inplace=True)
car_data.shape

(299, 9)

### 5. Drop the columns which you think redundant for the analysis.

In [14]:
car_data['Owner'].unique()

array([0, 1, 3])

In [15]:
len(car_data["Car_Name"].unique())

98

In [16]:
#car name looks like redundant it has 98 unique values
car_data = car_data.drop('Car_Name', axis=1)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [17]:
from datetime import date

current_year = date.today().year


In [18]:
car_data['Age_of_the_car']= current_year - car_data['Year']
car_data['Age_of_the_car'].head(5)

0     8
1     9
2     5
3    11
4     8
Name: Age_of_the_car, dtype: int64

In [19]:
car_data = car_data.drop('Year', axis=1)

### 7. Encode the categorical columns

In [20]:
categorical_features=car_data.select_dtypes(include=[np.object])
categorical_features.columns

Index(['Fuel_Type', 'Seller_Type', 'Transmission'], dtype='object')

In [21]:
# select object datatype variables
object_type_variables=[i for i in car_data[['Fuel_Type', 'Seller_Type', 'Transmission']] if car_data.dtypes[i]== object]
le=LabelEncoder()
def encoder(car_data):
    for i in object_type_variables:
        q=le.fit_transform(car_data[i].astype('str'))
        car_data[i]=q
        car_data[i]=car_data[i].astype('int')
encoder(car_data)

### 8. Separate the target and independent features.

In [22]:
X=car_data.drop('Selling_Price',axis=1)
y=car_data['Selling_Price']

### 9. Split the data into train and test.

In [23]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(239, 7) (60, 7)
(239,) (60,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [24]:

regressor = RandomForestRegressor(n_estimators = 25, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y)

RandomForestRegressor(n_estimators=25, random_state=0)

In [25]:
# Make prediction
predictions = regressor.predict(X_test)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)
r2

0.9924226856477791

### 11. Create a pickle file with an extension as .pkl

In [26]:
filename = '/Users/moni/Downloads/car_price_predictor.pkl'
pickle.dump(regressor, open(filename, 'wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
# created
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>

<body>

    <div class="hero-image">
      <div class="hero-text">

        <h1 style="font-size:50px">Used Car Price Predictor</h1>
         <br><br><h3>{{ prediction_text }}<h3>
      </div>
    </div>

     <style>

        body, html {
          height: 100%;
          margin: 0;
          font-family: Arial, Helvetica, sans-serif;
        }

        .hero-image {
          background-image: linear-gradient(rgba(0, 0, 0, 0.5), rgba(0, 0, 0, 0.5)), url('/static/image.jpg');
          height: 50%;
          background-position: bottom;
          background-repeat: no-repeat;
          background-size: cover;
          position: relative;
        }

        .hero-text {
          text-align: center;
          position: absolute;
          top: 50%;
          left: 50%;
          transform: translate(-50%, -50%);
          color: white;
        }

    </style>


    <div style="color:	rgb(0, 0, 0)">
        <form action="{{ url_for('predict')}}" method="post">
            <h2>Enter Car Details: </h2>
            <h3>Age of the car(In years)</h3>
            <input id="first" name="Age_of_the_car" type="number ">
            <h3>Present Showroom Price(In lakhs)</h3><br><input id="second" name="Present_Price" required="required">
            <h3>Kilometers Driven</h3><input id="third" name="Kms_Driven" required="required">
            <h3>Owner Type (0/1/3)</h3><br><input id="fourth" name="Owner" required="required">
            <h3>Fuel type</h3><br><select name="Fuel_Type" id="fuel" required="required">
                <option value="0">Petrol</option>
                <option value="1">Diesel</option>
                <option value="2">CNG</option>
            </select>
            <h3>Seller Type</h3><br><select name="Seller_Type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
            <h3>Transmission type</h3><br><select name="Transmission" id="research" required="required">
                <option value="0">Manual Car</option>
                <option value="1">Automatic Car</option>
            </select>
            <br><br><button id="sub" type="submit ">Predict Selling Price</button>
            <br>


        </form>

    </div>

    <style>
	body {
            background-color: 101, 10, 20;
            text-align: center;
            padding: 0px;
	    font-family: Helvetica;
        }

        #research {
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }

        #box {
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }

        #fuel {
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }

        #fuel:hover {
            background-color: white;
        }

        #research {
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #research:hover {
            background-color: white;
        }

        #resea {
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #resea:hover {
            background-color: white;
        }

        #sub {
            background-color: Green;
            font-family:'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }

        #sub:hover {
            background-color: white;
        }

        #first {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #second {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #third {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #fourth {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>

</html>

### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask,render_template,request,jsonify
import pickle
import numpy as np
import sklearn
app=Flask(__name__)
model=pickle.load(open('car_price_predictor.pkl','rb'))
@app.route('/',methods=['GET'])
def Home():
    return render_template('index.html')

@app.route("/predict",methods=['POST'])
def predict():
    if request.method == 'POST':
        Present_Price=float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_Type = request.form['Fuel_Type']
        Age_of_the_car =request.form['Age_of_the_car']
        Seller_Type =request.form['Seller_Type']
        Transmission=request.form['Transmission']
        prediction=model.predict([[Present_Price,Kms_Driven,Owner,Fuel_Type,Age_of_the_car,Seller_Type,Transmission]])
        output=round(prediction[0],2)
        return render_template('index.html',prediction_text="You can sell your car at {} lakhs".format(output))


if __name__=="__main__":
    app.run(debug=True)
        



### 13. Deploy your app on Heroku. (write commands for deployment)

In [ ]:
#commands
heroku login
heroku create carpricemrapp
echo >Procfile
git init
git add .
git commit -m "solution for car prediction"
heroku git:remote -a carpricemrapp
git push heroku master

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

In [ ]:
#url
https://carpricemrapp.herokuapp.com/

### Happy Learning :)